In [1]:
import os
import xarray as xr # For dealing with netCDF data
import pandas as pd
import netCDF4 as nc # Another way to deal with netCDF data
from datetime import timedelta, datetime
import netCDF4
from xarray import DataArray
import scipy.io
import matplotlib.pyplot as plt
import mat73
import numpy as np
import glob


print("done.")

done.


In [27]:
### open netCDF files
path="./LAPSE-RATE/DPLR1/"
path2="./LAPSE-RATE/DPLR2/"
path3="./LAPSE-RATE/CLMPS/"

ds1=xr.open_dataset(path + "UCB.DPLR1.a1.20180714.000000.cdf")   ### Saguache winds
ds2=xr.open_dataset(path + "UCB.DPLR1.a1.20180715.000000.cdf")
ds3=xr.open_dataset(path + "UCB.DPLR1.a1.20180716.000000.cdf")
ds4=xr.open_dataset(path + "UCB.DPLR1.a1.20180717.000000.cdf")
ds5=xr.open_dataset(path + "UCB.DPLR1.a1.20180718.000000.cdf")
ds6=xr.open_dataset(path + "UCB.DPLR1.a1.20180719.000000.cdf")
ds7=xr.open_dataset(path + "UCB.DPLR1.a1.20180720.000000.cdf")
ds_epsilon1=xr.open_dataset(path + "UCB.DPLR1.c1.20180713.233910.nc")  ## DPLR1_epsilon

ds8=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180714.000000.cdf")   ### Moffat winds (first file was at Saguache)
ds8=ds8.assign(latlon=(ds8.latlon))  ## Adding variables from attributes list
ds8=ds8.assign(msl=(ds8.msl))
ds8=ds8.assign(site=(ds8.site))
ds9=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180715.000000.cdf")
ds9=ds9.assign(latlon=(ds9.latlon))
ds9=ds9.assign(msl=(ds9.msl))
ds9=ds9.assign(site=(ds9.site))
ds10=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180716.000000.cdf")
ds10=ds10.assign(latlon=(ds10.latlon))
ds10=ds10.assign(msl=(ds10.msl))
ds10=ds10.assign(site=(ds10.site))
ds11=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180717.000000.cdf")
ds11=ds11.assign(latlon=(ds11.latlon))
ds11=ds11.assign(msl=(ds11.msl))
ds11=ds11.assign(site=(ds11.site))
ds12=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180718.000000.cdf")
ds12=ds12.assign(latlon=(ds12.latlon))
ds12=ds12.assign(msl=(ds12.msl))
ds12=ds12.assign(site=(ds12.site))
ds13=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180719.000000.cdf")
ds13=ds13.assign(latlon=(ds13.latlon))
ds13=ds13.assign(msl=(ds13.msl))
ds13=ds13.assign(site=(ds13.site))
ds14=xr.open_dataset(path2 + "UCB.DPLR2.a1.20180720.000000.cdf")
ds14=ds14.assign(latlon=(ds14.latlon))
ds14=ds14.assign(msl=(ds14.msl))
ds14=ds14.assign(site=(ds14.site))
ds_epsilon2=xr.open_dataset(path2 + "UCB.DPLR2.c1.20180713.233420.nc")  ## DPLR2_epsilon

ds15=xr.open_dataset(path3 + "UOK.CLMPS.DLVAD.c1.20180715.000214.cdf")   ### CLMPS winds
ds15=ds15.assign(time=((ds15["base_time"]+ds15['time_offset'])/10**9).astype(float))  ## converting from timedelta64[ns] to seconds since epoch
ds16=xr.open_dataset(path3 + "UOK.CLMPS.DLVAD.c1.20180716.000441.cdf")
ds16=ds16.assign(time=((ds16["base_time"]+ds16['time_offset'])/10**9).astype(float))
ds17=xr.open_dataset(path3 + "UOK.CLMPS.DLVAD.c1.20180717.000226.cdf")
ds17=ds17.assign(time=((ds17["base_time"]+ds17['time_offset'])/10**9).astype(float))
ds18=xr.open_dataset(path3 + "UOK.CLMPS.DLVAD.c1.20180718.000458.cdf")
ds18=ds18.assign(time=((ds18["base_time"]+ds18['time_offset'])/10**9).astype(float))
ds19=xr.open_dataset(path3 + "UOK.CLMPS.DLVAD.c1.20180719.000231.cdf")
ds19=ds19.assign(time=((ds19["base_time"]+ds19['time_offset'])/10**9).astype(float))
ds20=xr.open_dataset(path3 + "UOK.CLMPS.DLVAD.c1.20180720.000453.cdf")
ds20=ds20.assign(time=((ds20["base_time"]+ds20['time_offset'])/10**9).astype(float))
ds_epsilon3=xr.open_dataset(path3 + "UCB.CLMPS.c1.20180715.000017.nc")  ## CLMPS_epsilon

### combine winds into one netCDF for each instrument
DPLR1_combined = xr.concat([ds1,ds2,ds3,ds4,ds5,ds6,ds7],'time')
DPLR2_combined = xr.concat([ds9,ds8,ds10,ds11,ds12,ds13,ds14],'time')
CLMPS_combined = xr.concat([ds15,ds16,ds17,ds18,ds19,ds20],'time')

### for CLMPS, rename height dims (slanted/vertical beams) + add time since epoch attrs
CLMPS_combined = CLMPS_combined.rename_dims(height='slanted_beam')
CLMPS_combined = CLMPS_combined.rename(height='height_slanted')
ds_epsilon3 = ds_epsilon3.rename_dims(height='vertical_beam')
ds_epsilon3 = ds_epsilon3.rename(height='height_vertical')
CLMPS_combined.time.attrs['units']='sec'
CLMPS_combined.time.attrs['description']='seconds since epoch (00:00:00, 1 January 1970)'

### merge epsilon with winds
DPLR1_merged = xr.merge([DPLR1_combined,ds_epsilon1], combine_attrs='override')  ## (use 'override' to select attributes from first ds: https://xarray.pydata.org/en/stable/generated/xarray.merge.html)
DPLR2_merged = xr.merge([DPLR2_combined,ds_epsilon2], combine_attrs='override')
DPLR2_merged['latlon'][0:86]=np.array('38.09877$^{\\circ}$N, -106.17028$^{\\circ}$W', dtype='<U42')  ## Set first day latlon
DPLR2_merged['msl'][0:86]=np.array('2388 m', dtype='<U6')  ## Set first day msl
DPLR2_merged['site'][0:86]=np.array('Saguache', dtype='<U8')  ## Set first day site
CLMPS_merged = xr.merge([CLMPS_combined,ds_epsilon3], combine_attrs='no_conflicts')  ## (use 'no_conflicts' to retain unique attrs from both)

### save merged netCDF files
DPLR1_merged.to_netcdf(path + "UCB.DPLR1.d1.20180713.233910.cdf")   ### Saguache
DPLR2_merged.to_netcdf(path2 + "UCB.DPLR2.d1.20180713.233420.cdf") ### Moffat
CLMPS_merged.to_netcdf(path3 + "UCB.CLMPS.d1.20180715.000017.cdf")   ### CLMPS


print('done.')

/pl/active/courses/2020_fall/ATOC_BLM/software/anaconda/envs/tutorial_2020/lib/python3.7/site-packages/ipykernel_launcher.py:81: SerializationWarning: saving variable beam with floating point data as an integer dtype without any _FillValue to use for NaNs
/pl/active/courses/2020_fall/ATOC_BLM/software/anaconda/envs/tutorial_2020/lib/python3.7/site-packages/ipykernel_launcher.py:82: SerializationWarning: saving variable beam with floating point data as an integer dtype without any _FillValue to use for NaNs
/pl/active/courses/2020_fall/ATOC_BLM/software/anaconda/envs/tutorial_2020/lib/python3.7/site-packages/ipykernel_launcher.py:83: SerializationWarning: saving variable base_time with floating point data as an integer dtype without any _FillValue to use for NaNs


done.


In [31]:
DPLR1_merged

<xarray.Dataset>
Dimensions:        (height: 10, time: 769870)
Coordinates:
  * time           (time) float64 1.532e+09 1.532e+09 ... 1.532e+09 1.532e+09
  * height         (height) int32 40 60 80 100 120 140 160 180 200 220
Data variables:
    beam           (time) int32 ...
    wspd           (time, height) float32 ...
    wdir           (time, height) float32 ...
    CNR            (time, height) float32 ...
    RWS            (time, height) float32 ...
    RWSD           (time, height) float32 ...
    u              (time, height) float32 ...
    v              (time, height) float32 ...
    w              (time, height) float32 ...
    epsilon        (height, time) float64 ...
    epsilon_30min  (height, time) float64 ...
Attributes:
    description:                    CU profiling lidar observations from WC68
    source:                         Converted RTD files from Julie.Lundquist@...
    latlon:                         38.09877$^{\circ}$N, -106.17028$^{\circ}$W
    msl:                            2388 m
    site:                           Saguache
    GPS_Localisation:               GPS Localisation=38°05'55.57"N 106°10'13....
    ScanAngle:                      ScanAngle(°)=27.86
    Number_of_averaged_shots:       NumberOfAveragedShots=10000
    RollAngle:                      RollAngle(°)=0.40
    HeadingAngle:                   HeadingAngle(°)=-17.80
    Altitudes:                      Altitudes(m)= 40 60 80 100 120 140 160 18...
    Pitch_angle:                    PitchAngle(°)=-0.20
    Wavelength:                     Wavelength(nm)=1543.00
    Algorithm:                      Algorithm=MLE
    MAOfrequency:                   MAOFrequency(Hz)=68000000.000
    CutOfffrequency:                CutOffFrequency(Hz)=90000000.000
    WindowLength:                   WindowLength(s)=0.000000E+0
    IndexRange:                     IndexRange0=70
    PathLaserPulse:                 PathLaserPulse=C:\Program Files\Leosphere...
    NumberOfSamplesLaserPulse:      NumberOfSamples_LaserPulse=74
    BeginIndex_LaserPulse:          BeginIndex_LaserPulse=0
    BeginIndex_NoiseReference:      BeginIndex_NoiseReference=0
    NumberOfNoiseWindowsByProfile:  NumberOfNoiseWindowsByProfile=1
    VrThreshold:                    VrThreshold (m/s)=)1.700
    SigmaFreqThreshold:             SigmaFreqThreshold (m/s)=)0.750
    SamplingInterval:               SamplingInterval=4.000000E-9
    NumberOfSamples:                NumberOfSamples=1024
    VerticalCoupling:               VerticalCoupling=AC, 50 Ohm
    Offset:                         Offset=0.000
    FullScale:                      FullScale=0.200
    NumberOfSegments:               NumberOfSegments=100
    TriggerSource:                  TriggerSource=External
    TriggerCoupling:                TriggerCoupling=DC, 50 Ohm (Ext. trig. only)
    TriggerSlope:                   TriggerSlope=Positive
    TriggerLevel1:                  TriggerLevel1=3000
    TriggerLevel2:                  TriggerLevel2=0
    DelayLine:                      DelayLine=4.000000E-7
    AbsoluteInitialPosition:        AbsoluteInitialPosition(°)=122040
    RelativeInitialPosition:        RelativeInitialPosition(°)=0
    InitialDateTime:                InitialDate/Time=14/07/2018 00:00:00.68
    Gain:                           Gain=32.797000
    InitialLD1:                     InitialLD1(mA)=IC1=320
    InitialLD2:                     InitialLD2(mA)=IC2=1650

In [32]:
DPLR2_merged

<xarray.Dataset>
Dimensions:        (height: 10, time: 743449)
Coordinates:
  * time           (time) float64 1.532e+09 1.532e+09 ... 1.532e+09 1.532e+09
  * height         (height) int32 40 60 80 100 120 140 160 180 200 220
Data variables:
    beam           (time) int32 ...
    wspd           (time, height) float32 ...
    wdir           (time, height) float32 ...
    CNR            (time, height) float32 ...
    RWS            (time, height) float32 ...
    RWSD           (time, height) float32 ...
    u              (time, height) float32 ...
    v              (time, height) float32 ...
    w              (time, height) float32 ...
    latlon         (time) object ...
    msl            (time) object ...
    site           (time) object ...
    epsilon        (height, time) float64 ...
    epsilon_30min  (height, time) float64 ...
Attributes:
    description:                    CU profiling lidar observations from WC49
    source:                         Converted RTD files from Julie.Lundquist@...
    latlon:                         37.99794$^{\circ}$N, -105.91199$^{\circ}$W
    msl:                            2308 m
    site:                           Moffat School
    GPS_Localisation:               GPS Localisation=Not Available
    ScanAngle:                      ScanAngle(°)=27.85
    Number_of_averaged_shots:       NumberOfAveragedShots=10000
    RollAngle:                      RollAngle(°)=0.10
    HeadingAngle:                   HeadingAngle(°)=-18.50
    Altitudes:                      Altitudes(m)= 40 60 80 100 120 140 160 18...
    Pitch_angle:                    PitchAngle(°)=0.10
    Wavelength:                     Wavelength(nm)=1543.00
    Algorithm:                      Algorithm=MLE
    MAOfrequency:                   MAOFrequency(Hz)=68014368.780
    CutOfffrequency:                CutOffFrequency(Hz)=90000000.000
    WindowLength:                   WindowLength(s)=0.000000E+0
    IndexRange:                     IndexRange0=62
    PathLaserPulse:                 PathLaserPulse=C:\Documents and Settings\...
    NumberOfSamplesLaserPulse:      NumberOfSamples_LaserPulse=74
    BeginIndex_LaserPulse:          BeginIndex_LaserPulse=0
    BeginIndex_NoiseReference:      BeginIndex_NoiseReference=0
    NumberOfNoiseWindowsByProfile:  NumberOfNoiseWindowsByProfile=1
    VrThreshold:                    VrThreshold (m/s)=)1.700
    SigmaFreqThreshold:             SigmaFreqThreshold (m/s)=)0.750
    SamplingInterval:               SamplingInterval=4.000000E-9
    NumberOfSamples:                NumberOfSamples=1024
    VerticalCoupling:               VerticalCoupling=AC, 50 Ohm
    Offset:                         Offset=0.000
    FullScale:                      FullScale=0.200
    NumberOfSegments:               NumberOfSegments=100
    TriggerSource:                  TriggerSource=External
    TriggerCoupling:                TriggerCoupling=DC, 50 Ohm (Ext. trig. only)
    TriggerSlope:                   TriggerSlope=Positive
    TriggerLevel1:                  TriggerLevel1=3000
    TriggerLevel2:                  TriggerLevel2=0
    DelayLine:                      DelayLine=4.000000E-7
    AbsoluteInitialPosition:        AbsoluteInitialPosition(°)=71820
    RelativeInitialPosition:        RelativeInitialPosition(°)=180
    InitialDateTime:                InitialDate/Time=15/07/2018 00:00:00.56
    Gain:                           Gain=32.797000
    InitialLD1:                     InitialLD1(mA)=IC1=320
    InitialLD2:                     InitialLD2(mA)=IC2=2100

In [33]:
CLMPS_merged

<xarray.Dataset>
Dimensions:          (slanted_beam: 320, time: 322494, vertical_beam: 320)
Coordinates:
  * time             (time) float64 1.532e+09 1.532e+09 ... 1.532e+09 1.532e+09
    height_slanted   (slanted_beam) float32 ...
    height_vertical  (vertical_beam) float64 ...
Dimensions without coordinates: slanted_beam, vertical_beam
Data variables:
    base_time        (time) timedelta64[ns] ...
    time_offset      (time) timedelta64[ns] ...
    hour             (time) timedelta64[ns] ...
    wspd             (time, slanted_beam) float32 ...
    wdir             (time, slanted_beam) float32 ...
    w                (time, slanted_beam) float32 ...
    rms              (time, slanted_beam) float32 ...
    intensity        (time, slanted_beam) float32 ...
    lat              (time) float32 ...
    lon              (time) float32 ...
    alt              (time) float32 ...
    epsilon          (vertical_beam, time) float64 ...
    epsilon_30min    (vertical_beam, time) float64 ...
Attributes:
    Date_file_created:  Mon Jul 16 18:56:48 2018
    Data_source:        CLAMPS Doppler Lidar
    Data_contact:       Petra Klein (pkklein@ou.edu) and Dave Turner (dave.tu...
    Direction_offset:      0.0 degrees
    description:        Turbulence dissipation rate estimated from UOK Halo l...
    lat-lon:            37.99759N, -105.9118W
    elevation (msl):    2305 m
    site:               Moffat School